# Investment Recommendation System

## Overview
This notebook implements a comprehensive investment recommendation system that analyzes both stocks and mutual funds to provide personalized investment recommendations. The system uses multiple machine learning models for price prediction, risk analysis, and portfolio optimization.

## Features
1. Data Processing and Analysis
   - Stock data analysis (12 stocks)
   - Mutual fund data analysis (6 funds)
   - Technical indicators
   - Feature engineering

2. Machine Learning Models
   - LSTM for short-term predictions
   - ARIMA-GARCH for medium-term predictions
   - Prophet for long-term forecasting

3. Risk Analysis
   - Volatility calculation
   - Sharpe ratio
   - Risk classification
   - Portfolio risk assessment

4. Portfolio Optimization
   - Modern Portfolio Theory
   - Risk-adjusted allocation
   - Balanced portfolio strategy

5. Results and Reporting
   - Price predictions
   - Risk profiles
   - Portfolio recommendations
   - Visual reports

   ## Prerequisites
   - Python 3.8+
   - Google Colab with GPU runtime
   - Required files:
     - 12 stock files (stock_1.csv/xlsx to stock_12.csv/xlsx)
     - 6 mutual fund files (mf_1.csv/xlsx to mf_6.csv/xlsx)

Initial setup

1.0 GPU Check

In [ ]:
# Replace the GPU check and installation block with:
# Check GPU availability
import tensorflow as tf
print("GPU Available:", tf.config.list_physical_devices('GPU'))

# Install packages with compatible versions
%pip install numpy pandas scikit-learn tensorflow keras yfinance ta prophet pmdarima arch matplotlib seaborn plotly openpyxl

# Add GPU memory management
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    for gpu in gpus:
        tf.config.experimental.set_memory_growth(gpu, True)

Section 1: Setup and Dependencies

In [38]:
# Install required packages
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.graph_objects as go
from datetime import datetime, timedelta
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from prophet import Prophet
from arch import arch_model
import ta
import warnings
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error
warnings.filterwarnings('ignore')

# Set random seed for reproducibility
np.random.seed(42)

In [ ]:
# Import Local Data Section
import os
from google.colab import files  # For Google Colab

# Create necessary directories
os.makedirs('data/raw', exist_ok=True)
os.makedirs('data/processed', exist_ok=True)

def upload_and_process_files():
    """Upload files and categorize them based on their content"""
    print("Please upload your stock and mutual fund files (CSV or Excel)...")
    uploaded = files.upload()  # This opens a file picker dialog
    
    stock_data = {}
    mf_data = {}
    
    for filename in uploaded.keys():
        # Save the uploaded file
        file_path = f'data/raw/{filename}'
        with open(file_path, 'wb') as f:
            f.write(uploaded[filename])
        
        try:
            # Check file extension to determine how to read it
            if filename.lower().endswith('.xlsx') or filename.lower().endswith('.xls'):
                df = pd.read_excel(file_path)
                print(f"Reading as Excel file: {filename}")
            else:
                df = pd.read_csv(file_path)
                print(f"Reading as CSV file: {filename}")
            
            # Convert Date column to datetime if present
            if 'Date' in df.columns:
                df['Date'] = pd.to_datetime(df['Date'])
            
            # For mutual funds, look for NAV column
            if 'NAV' in df.columns:
                mf_data[filename] = df
                print(f"✅ Processed as MUTUAL FUND data: {filename}")
                continue
                
            # For stocks, check for typical stock columns
            stock_cols = ['Open', 'High', 'Low', 'Close']
            if any(col in df.columns for col in stock_cols):
                stock_data[filename] = df
                print(f"✅ Processed as STOCK data: {filename}")
                continue
                
            # If we reach here, we couldn't categorize the file
            print(f"❓ Couldn't automatically categorize {filename}")
            print(f"\nColumns in {filename}: {df.columns.tolist()}")
            
            # In Colab we can't use input(), so we'll make a best guess
            if 'Volume' in df.columns:
                stock_data[filename] = df
                print(f"✅ Guessed as STOCK data (has Volume column): {filename}")
            else:
                # Default to MF if we can't determine
                mf_data[filename] = df
                print(f"✅ Guessed as MUTUAL FUND data: {filename}")
                
        except Exception as e:
            print(f"❌ Error processing {filename}: {str(e)}")
            print(f"This might be due to file format issues. Check if {filename} is properly formatted.")
    
    print(f"\nSummary: Loaded {len(stock_data)} stock files and {len(mf_data)} mutual fund files")
    return stock_data, mf_data

# Run the upload and categorization
stock_data, mf_data = upload_and_process_files()

# Display sample of loaded data
if stock_data:
    print("\n📊 Sample of Stock Data:")
    sample_stock = list(stock_data.values())[0]
    file_name = list(stock_data.keys())[0]
    print(f"From file: {file_name}")
    display(sample_stock.head())
    print(f"Columns: {sample_stock.columns.tolist()}")

if mf_data:
    print("\n📊 Sample of Mutual Fund Data:")
    sample_mf = list(mf_data.values())[0]
    file_name = list(mf_data.keys())[0]
    print(f"From file: {file_name}")
    display(sample_mf.head())
    print(f"Columns: {sample_mf.columns.tolist()}")

Section 2: Data Processing Class

In [ ]:
class DataProcessor:
    def prepare_lstm_data(self, df, sequence_length=60):
        # Add critical validation
        if len(df) < sequence_length:
            raise ValueError(f"Data length ({len(df)}) is less than sequence length ({sequence_length})")
        
        if df.isnull().any().any():
            raise ValueError("Data contains NaN values. Please clean the data first.")
        
    def __init__(self):
        self.stock_scaler = MinMaxScaler()
        self.mf_scaler = MinMaxScaler()
        
    def load_stock_data(self, file_path):
        """Load stock data from CSV file."""
        df = pd.read_csv(file_path)
        df['Date'] = pd.to_datetime(df['Date'])
        df = df.sort_values('Date')
        return df
    
    def load_mutual_fund_data(self, file_path):
        """Load mutual fund data from CSV file."""
        df = pd.read_csv(file_path)
        df['Date'] = pd.to_datetime(df['Date'])
        df = df.sort_values('Date')
        return df
    
    def add_technical_indicators(self, df):
        """Add technical indicators to the dataset."""
        # RSI
        df['RSI'] = ta.momentum.RSIIndicator(df['price']).rsi()
        
        # MACD
        macd = ta.trend.MACD(df['price'])
        df['MACD'] = macd.macd()
        df['MACD_Signal'] = macd.macd_signal()
        
        # Bollinger Bands
        bollinger = ta.volatility.BollingerBands(df['price'])
        df['BB_High'] = bollinger.bollinger_hband()
        df['BB_Low'] = bollinger.bollinger_lband()
        
        # Volume indicators
        df['Volume_MA'] = ta.trend.SMAIndicator(df['Volume'], window=20).sma_indicator()
        
        return df
    
    def prepare_lstm_data(self, df, sequence_length=60):
        """Prepare data for LSTM model training."""
        # Select features for LSTM
        features = ['open', 'High', 'Low', 'price', 'Volume', 'RSI', 'MACD', 'MACD_Signal']
        data = df[features].values
        
        # Scale the data
        scaled_data = self.stock_scaler.fit_transform(data)
        
        # Create sequences
        X, y = [], []
        for i in range(len(scaled_data) - sequence_length):
            X.append(scaled_data[i:(i + sequence_length)])
            y.append(scaled_data[i + sequence_length, 3])  # Predict price
            
        return np.array(X), np.array(y)
    
    def prepare_mf_lstm_data(self, df, sequence_length=60):
        """Prepare mutual fund data for LSTM model training."""
        # Select features for LSTM
        features = ['NAV']
        data = df[features].values
        
        # Scale the data
        scaled_data = self.mf_scaler.fit_transform(data)
        
        # Create sequences
        X, y = [], []
        for i in range(len(scaled_data) - sequence_length):
            X.append(scaled_data[i:(i + sequence_length)])
            y.append(scaled_data[i + sequence_length, 0])  # Predict NAV
            
        return np.array(X), np.array(y)
    
    def prepare_prophet_data(self, df, is_mf=False):
        """Prepare data for Prophet model."""
        if is_mf:
            prophet_data = df[['Date', 'NAV']].copy()
            prophet_data.columns = ['ds', 'y']
        else:
            prophet_data = df[['Date', 'price']].copy()
            prophet_data.columns = ['ds', 'y']
        return prophet_data


In [21]:
def standardize_column_names(df):
    """Standardize column names across different file formats."""
    column_mapping = {
        'Close': 'Price',
        'Adj Close': 'Price',
        'Net Asset Value': 'NAV',
        'Net Asset Value (NAV)': 'NAV'
    }
    df.columns = [column_mapping.get(col, col) for col in df.columns]
    return df

data cleaning method

In [22]:
def clean_data(self, df):
   """Clean and preprocess data."""
   # Remove duplicates
   df = df.drop_duplicates()
   # Forward fill missing values
   df = df.fillna(method='ffill')
   # Backward fill any remaining missing values
   df = df.fillna(method='bfill')
   return df

Section 3: Model Training Class

In [23]:
class ModelTrainer:
    def train_lstm(self, X_train, y_train, X_val, y_val):
        # Add critical validation
        if X_train.shape[0] == 0 or y_train.shape[0] == 0:
            raise ValueError("Empty training data")
            
        if X_train.shape[1] != X_val.shape[1]:
            raise ValueError("Training and validation data have different feature dimensions")
            
        # Add early stopping to prevent overfitting
        early_stopping = tf.keras.callbacks.EarlyStopping(
            monitor='val_loss',
            patience=10,
            restore_best_weights=True
        )
    def __init__(self):
        self.stock_models = {}
        self.mf_models = {}
        
    def create_lstm_model(self, input_shape):
        """Create and compile LSTM model."""
        model = Sequential([
            LSTM(50, return_sequences=True, input_shape=input_shape),
            Dropout(0.2),
            LSTM(50, return_sequences=False),
            Dropout(0.2),
            Dense(1)
        ])
        
        model.compile(optimizer='adam', loss='mse')
        return model
    
    def train_lstm(self, X_train, y_train, X_val, y_val):
        """Train LSTM model."""
        model = self.create_lstm_model((X_train.shape[1], X_train.shape[2]))
        
        history = model.fit(
            X_train, y_train,
            epochs=50,
            batch_size=32,
            validation_data=(X_val, y_val),
            verbose=1
        )
        
        return model, history
    
    def train_prophet(self, data):
        """Train Prophet model."""
        model = Prophet(daily_seasonality=True)
        model.fit(data)
        return model
    
    def train_arima_garch(self, data):
        """Train ARIMA-GARCH model."""
        # Fit ARIMA model
        arima = arch.arch_model(data, vol='Garch', p=1, q=1)
        results = arima.fit(disp='off')
        return results

Section 4: Risk Analysis Class


In [24]:
class RiskAnalyzer:
    def analyze_stock_risk(self, data):
        # Add critical validation
        if data.empty:
            raise ValueError("Empty data provided for risk analysis")
            
        if 'price' not in data.columns:
            raise ValueError("Price column not found in data")
            
    def __init__(self):
        self.stock_risk_metrics = {}
        self.mf_risk_metrics = {}
        
    def calculate_volatility(self, returns, window=252):
        """Calculate annualized volatility."""
        return returns.std() * np.sqrt(252)
    
    def calculate_sharpe_ratio(self, returns, risk_free_rate=0.02):
        """Calculate Sharpe ratio."""
        excess_returns = returns - risk_free_rate/252
        return np.sqrt(252) * excess_returns.mean() / excess_returns.std()
    
    def calculate_max_drawdown(self, prices):
        """Calculate maximum drawdown."""
        peak = prices.expanding(min_periods=1).max()
        drawdown = (prices - peak) / peak
        return drawdown.min()
    
    def classify_risk(self, volatility):
        """Classify risk based on volatility."""
        if volatility < 0.15:
            return 'Low'
        elif volatility < 0.30:
            return 'Medium'
        else:
            return 'High'
    
    def analyze_stock_risk(self, data):
        """Analyze risk for stock data."""
        returns = data['price'].pct_change().dropna()
        
        self.stock_risk_metrics['volatility'] = self.calculate_volatility(returns)
        self.stock_risk_metrics['sharpe_ratio'] = self.calculate_sharpe_ratio(returns)
        self.stock_risk_metrics['max_drawdown'] = self.calculate_max_drawdown(data['price'])
        self.stock_risk_metrics['risk_category'] = self.classify_risk(self.stock_risk_metrics['volatility'])
        
        return self.stock_risk_metrics
    
    def analyze_mf_risk(self, data):
        """Analyze risk for mutual fund data."""
        returns = data['NAV'].pct_change().dropna()
        
        self.mf_risk_metrics['volatility'] = self.calculate_volatility(returns)
        self.mf_risk_metrics['sharpe_ratio'] = self.calculate_sharpe_ratio(returns)
        self.mf_risk_metrics['max_drawdown'] = self.calculate_max_drawdown(data['NAV'])
        self.mf_risk_metrics['risk_category'] = self.classify_risk(self.mf_risk_metrics['volatility'])
        
        return self.mf_risk_metrics

Section 5: Portfolio Optimization Class


In [25]:
class PortfolioOptimizer:
    def optimize_portfolio(self, returns, num_portfolios=10000):
        # Add critical validation
        if returns.empty:
            raise ValueError("No returns data provided for portfolio optimization")
            
        if len(returns) < 252:  # Minimum for annual calculations
            raise ValueError("Insufficient data for portfolio optimization. Need at least 252 days of data.")
            
    def __init__(self):
        self.stock_weights = None
        self.mf_weights = None
        self.returns = None
        self.risk = None
        
    def calculate_portfolio_metrics(self, returns, weights):
        """Calculate portfolio returns and risk."""
        portfolio_returns = np.sum(returns.mean() * weights) * 252
        portfolio_risk = np.sqrt(np.dot(weights.T, np.dot(returns.cov() * 252, weights)))
        return portfolio_returns, portfolio_risk
    
    def optimize_portfolio(self, returns, num_portfolios=10000):
        """Optimize portfolio weights using Monte Carlo simulation."""
        num_assets = len(returns.columns)
        results = np.zeros((3, num_portfolios))
        weights_list = []
        
        for i in range(num_portfolios):
            weights = np.random.random(num_assets)
            weights /= np.sum(weights)
            weights_list.append(weights)
            
            portfolio_returns, portfolio_risk = self.calculate_portfolio_metrics(returns, weights)
            results[0, i] = portfolio_returns
            results[1, i] = portfolio_risk
            results[2, i] = portfolio_returns / portfolio_risk  # Sharpe ratio
            
        # Find optimal portfolio
        optimal_idx = np.argmax(results[2])
        self.weights = weights_list[optimal_idx]
        self.returns = results[0, optimal_idx]
        self.risk = results[1, optimal_idx]
        
        return self.weights, self.returns, self.risk
    
    def optimize_balanced_portfolio(self, stock_returns, mf_returns, stock_weight=0.6):
        """Optimize a balanced portfolio with both stocks and mutual funds."""
        # Optimize stock portfolio
        stock_weights, stock_returns, stock_risk = self.optimize_portfolio(stock_returns)
        
        # Optimize mutual fund portfolio
        mf_weights, mf_returns, mf_risk = self.optimize_portfolio(mf_returns)
        
        # Combine portfolios
        combined_returns = stock_weight * stock_returns + (1 - stock_weight) * mf_returns
        combined_risk = np.sqrt(stock_weight**2 * stock_risk**2 + (1-stock_weight)**2 * mf_risk**2)
        
        return {
            'stock_weights': stock_weights,
            'mf_weights': mf_weights,
            'stock_weight': stock_weight,
            'combined_returns': combined_returns,
            'combined_risk': combined_risk
        }

Section 6: Main Execution


In [ ]:
# Add error handling for model training
try:
    # Initialize components
    data_processor = DataProcessor()
    model_trainer = ModelTrainer()
    risk_analyzer = RiskAnalyzer()
    portfolio_optimizer = PortfolioOptimizer()
    
    # Process data
    for file, data in stock_data.items():
        try:
            # Prepare data
            X, y = data_processor.prepare_lstm_data(data)
            
            # Train model
            model, history = model_trainer.train_lstm(X_train, y_train, X_val, y_val)
            
        except Exception as e:
            print(f"Error processing {file}: {str(e)}")
            continue
            
except Exception as e:
    print(f"Fatal error in main execution: {str(e)}")
    raise

Section 7: Data Loading

In [ ]:
# Load and preprocess stock data
def upload_and_process_files():
    # Add critical validation
    if not uploaded:
        raise ValueError("No files were uploaded. Please upload your data files.")
        
    for filename, df in stock_data.items():
        if len(df) < 60:  # Minimum required for LSTM sequence
            raise ValueError(f"Insufficient data points in {filename}. Need at least 60 data points.")
            
print("Loading stock data...")
stock_data = {}
for file in stock_files:
    try:
        data = data_processor.load_stock_data(file)
        data = data_processor.add_technical_indicators(data)
        stock_data[file] = data
        print(f"Loaded {file}")
    except Exception as e:
        print(f"Error loading {file}: {str(e)}")

# Load and preprocess mutual fund data
print("\nLoading mutual fund data...")
mf_data = {}
for file in mf_files:
    try:
        data = data_processor.load_mutual_fund_data(file)
        mf_data[file] = data
        print(f"Loaded {file}")
    except Exception as e:
        print(f"Error loading {file}: {str(e)}")

7.1 Data Summary

In [ ]:
# Print data summary
print("\nData Summary:")
print("\nStock Data:")
for file, data in stock_data.items():
    print(f"\n{file}:")
    print(f"Shape: {data.shape}")
    print(f"Date Range: {data['Date'].min()} to {data['Date'].max()}")
    print(f"Missing Values: {data.isnull().sum().sum()}")

print("\nMutual Fund Data:")
for file, data in mf_data.items():
    print(f"\n{file}:")
    print(f"Shape: {data.shape}")
    print(f"Date Range: {data['Date'].min()} to {data['Date'].max()}")
    print(f"Missing Values: {data.isnull().sum().sum()}")

Section 8: Model Training


In [ ]:
# Train models for stocks
print("\nTraining stock models...")
for file, data in stock_data.items():
    print(f"\nTraining models for {file}")
    
    # Prepare data
    X, y = data_processor.prepare_lstm_data(data)
    
    # Split data
    train_size = int(len(X) * 0.8)
    X_train, X_val = X[:train_size], X[train_size:]
    y_train, y_val = y[:train_size], y[train_size:]
    
    # Train LSTM model
    model, history = model_trainer.train_lstm(X_train, y_train, X_val, y_val)
    model_trainer.stock_models[file] = model
    
    # Plot training history
    plt.figure(figsize=(10, 6))
    plt.plot(history.history['loss'], label='Training Loss')
    plt.plot(history.history['val_loss'], label='Validation Loss')
    plt.title(f'Model Loss for {file}')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend()
    plt.show()

Section 9: Risk Analysis


In [ ]:
# Perform risk analysis
print("\nPerforming risk analysis...")
stock_risk_profiles = {}
mf_risk_profiles = {}

for file, data in stock_data.items():
    risk_metrics = risk_analyzer.analyze_stock_risk(data)
    stock_risk_profiles[file] = risk_metrics
    print(f"\nRisk Profile for {file}:")
    for metric, value in risk_metrics.items():
        print(f"{metric}: {value}")

for file, data in mf_data.items():
    risk_metrics = risk_analyzer.analyze_mf_risk(data)
    mf_risk_profiles[file] = risk_metrics
    print(f"\nRisk Profile for {file}:")
    for metric, value in risk_metrics.items():
        print(f"{metric}: {value}")

Section 10: Portfolio Optimization


In [ ]:
# Optimize portfolio
print("\nOptimizing portfolio...")

# Calculate returns for portfolio optimization
stock_returns = pd.DataFrame()
for file, data in stock_data.items():
    stock_returns[file] = data['price'].pct_change()

mf_returns = pd.DataFrame()
for file, data in mf_data.items():
    mf_returns[file] = data['NAV'].pct_change()

stock_returns = stock_returns.dropna()
mf_returns = mf_returns.dropna()

# Optimize balanced portfolio
portfolio_results = portfolio_optimizer.optimize_balanced_portfolio(stock_returns, mf_returns)

# Display results
print("\nOptimal Portfolio Allocation:")
print("\nStocks:")
for file, weight in zip(stock_files, portfolio_results['stock_weights']):
    print(f"{file}: {weight:.2%}")

print("\nMutual Funds:")
for file, weight in zip(mf_files, portfolio_results['mf_weights']):
    print(f"{file}: {weight:.2%}")

print(f"\nStock Allocation: {portfolio_results['stock_weight']:.2%}")
print(f"Mutual Fund Allocation: {(1-portfolio_results['stock_weight']):.2%}")
print(f"\nExpected Portfolio Return: {portfolio_results['combined_returns']:.2%}")
print(f"Portfolio Risk: {portfolio_results['combined_risk']:.2%}")

Section 11: Visualization


In [ ]:
# Visualize portfolio allocation
plt.figure(figsize=(15, 5))

# Stock allocation
plt.subplot(1, 2, 1)
plt.pie(portfolio_results['stock_weights'], labels=stock_files, autopct='%1.1f%%')
plt.title('Stock Portfolio Allocation')

# Mutual fund allocation
plt.subplot(1, 2, 2)
plt.pie(portfolio_results['mf_weights'], labels=mf_files, autopct='%1.1f%%')
plt.title('Mutual Fund Portfolio Allocation')

plt.tight_layout()
plt.show()

Section 12: Generate Predictions

In [ ]:
# Generate predictions
print("\nGenerating predictions...")

stock_predictions = {}
mf_predictions = {}

# Stock predictions
for file, data in stock_data.items():
    # Prepare data for prediction
    X, _ = data_processor.prepare_lstm_data(data)
    last_sequence = X[-1:]
    
    # Generate predictions
    model = model_trainer.stock_models[file]
    predicted_scaled = model.predict(last_sequence)
    
    # Inverse transform predictions
    predicted_price = data_processor.stock_scaler.inverse_transform(
        np.hstack([np.zeros((1, X.shape[2]-1)), predicted_scaled])
    )[:, 3]
    
    stock_predictions[file] = predicted_price[0]
    print(f"Predicted price for {file}: ₹{predicted_price[0]:.2f}")

# Mutual fund predictions
for file, data in mf_data.items():
    # Prepare data for prediction
    X, _ = data_processor.prepare_mf_lstm_data(data)
    last_sequence = X[-1:]
    
    # Generate predictions
    model = model_trainer.mf_models[file]
    predicted_scaled = model.predict(last_sequence)
    
    # Inverse transform predictions
    predicted_nav = data_processor.mf_scaler.inverse_transform(predicted_scaled)
    
    mf_predictions[file] = predicted_nav[0][0]
    print(f"Predicted NAV for {file}: ₹{predicted_nav[0][0]:.2f}")

13. Save to Google Drive

In [ ]:
# Create results directory if it doesn't exist
!mkdir -p results

# Save stock portfolio allocation
stock_allocation = pd.DataFrame({
    'Ticker': stock_files,
    'Weight': portfolio_results['stock_weights'],
    'Risk_Category': [stock_risk_profiles[file]['risk_category'] for file in stock_files],
    'Predicted_Price': [stock_predictions[file] for file in stock_files]
})
stock_allocation.to_csv('results/stock_portfolio_allocation.csv', index=False)

# Save mutual fund portfolio allocation
mf_allocation = pd.DataFrame({
    'Fund': mf_files,
    'Weight': portfolio_results['mf_weights'],
    'Risk_Category': [mf_risk_profiles[file]['risk_category'] for file in mf_files],
    'Predicted_NAV': [mf_predictions[file] for file in mf_files]
})
mf_allocation.to_csv('results/mf_portfolio_allocation.csv', index=False)

# Save portfolio summary
portfolio_summary = pd.DataFrame({
    'Metric': ['Stock_Allocation', 'Mutual_Fund_Allocation', 'Expected_Return', 'Portfolio_Risk'],
    'Value': [
        portfolio_results['stock_weight'],
        1 - portfolio_results['stock_weight'],
        portfolio_results['combined_returns'],
        portfolio_results['combined_risk']
    ]
})
portfolio_summary.to_csv('results/portfolio_summary.csv', index=False)

print("\nResults saved to 'results' directory:")
print("1. stock_portfolio_allocation.csv")
print("2. mf_portfolio_allocation.csv")
print("3. portfolio_summary.csv")

13.1 Results Verification


In [ ]:
# Verify results directory contents
print("\nVerifying results directory contents:")
!ls -l results/

Section 14: Mount Google Drive and save everything

In [ ]:
# Mount Google Drive and save everything
from google.colab import drive
from datetime import datetime

# Mount Google Drive
drive.mount('/content/drive')

# Create timestamped folder
timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
folder_name = f'Investment_Recommendation_{timestamp}'
!mkdir -p /content/drive/MyDrive/{folder_name}

# Save notebook
!cp /content/Investment_Recommendation_System.ipynb /content/drive/MyDrive/{folder_name}/

# Save results
!cp -r results /content/drive/MyDrive/{folder_name}/

# Save models (if you saved them)
!cp -r models /content/drive/MyDrive/{folder_name}/

print(f"\nAll files saved to Google Drive folder: {folder_name}")